In [42]:
#pip show plotly jupyterlab_widgets panel ipywidgets
#Name: plotly Version: 6.0.0
#Name: jupyterlab_widgets Version: 3.0.13
#Name: panel Version: 1.6.0
#Name: ipywidgets Version: 8.1.5
#Name: anywidget Version: 0.9.13
#

#https://chart-studio.plotly.com/~empet/15765/animating-a-3d-orthonormal-basis-http/#/
#https://community.plotly.com/t/how-to-use-plotly-python-for-complex-3d-animations/32983/6
#Note: change "Windowing mode" to 'defer': Improve loading time - Wait for idle CPU cycles
#to attach out of viewport cell, to stop notepad flickering up and down.
#
import plotly.graph_objs as go
import numpy as np
from numpy import pi, sin, cos, sqrt
import panel as pn

pn.extension("plotly")

def spiral(t, r=1, a=3):
    return r*cos(t), r*sin(t), a*t


def tangent(s, r=1, a=3):
    return np.array([-r*sin(s), r*cos(s), a])/sqrt(r**2+a**2)

def binormal(s, r=1, a=3):
    return np.array([a*r*sin(s), -a*r*cos(s), 1])/sqrt(a**2*r**2+1)

def get_frame_data(s,   r=1, a=3, f=1):
    p = spiral(s)
    u1 = tangent(s)
    u3 = binormal(s)
    u2 = np.cross(u3, u1)
    # for visualization purposes we are multiplying the base unit vectors by a factor f
    u1 = f*u1
    u2 = f*u2
    u3 = f*u3
    q = [p+u1, p+u2, p+u3]
    
    #The orthonormal vectors are defined in a single Scatter3d trace, not in three traces. 
    #We are separating by `None` the coordinates of thwo consecutive vectors.
    
    return go.Scatter3d(x = [p[0], q[0][0], None, p[0], q[1][0], None, p[0], q[2][0]],
                        y = [p[1], q[0][1], None, p[1], q[1][1], None, p[1], q[2][1]],
                        z = [p[2], q[0][2], None, p[2], q[1][2], None, p[2], q[2][2]])
    

n_frames = 38 #number of frames
t = np.linspace(0, 3*pi, n_frames)  #parameter values where the spiral and Frenet frames will be evaluated
xs, ys, zs = spiral(t)

fig = go.Figure(go.Scatter3d(x=xs, y=ys, z=zs, mode='lines', line_width=4))  #spiral trace

vec_fac = 1 #scaling factor for the three vectors
fig.add_trace(get_frame_data(0, f=vec_fac)) #add the orthonormal basis at the spiral point corresponding to s=0
fig.data[1].update(mode='lines', line_color='red', line_width=7) # update the basis trace by setting vector color, etc

#update layout attributes:
fig.update_scenes( 
                  #xaxis_visible=False, 
                  #yaxis_visible=False, 
                  #zaxis_visible=False,
    
                  #axis autorange=False, below, ensures that axes range  don't enlarge or  
                  #shrink,  according to frame data  (to see its effect, just comment the next three lines)
                  xaxis_autorange=False,
                  yaxis_autorange=False,
                  zaxis_autorange=False,
                  xaxis_range=[xs.min()-1, xs.max()+1],
                  yaxis_range=[ys.min()-1, ys.max()+1],
                  zaxis_range=[zs.min()-1, zs.max()+1], 
                  camera_eye=dict(x=1.2, y=1.2, z=0.8), # camera position
                  #camera_projection_type='orthographic'    #thedefault projection is perspective
)
fig.update_layout(width=800, height=600);
#fig.show()



frames = []

#for  s in t:
#    frames.append(go.Frame(data=[get_frame_data(s, f=vec_fac)],
#                           traces=[1])) #traces=[1] tells Plotly.js that each frame updates fig.data[1], i.e. the vectors

for  k, s in enumerate(t):
    frames.append(go.Frame(data=[get_frame_data(s, f=vec_fac)],
                           name = f'fr{k}',
                           traces=[1])) 
    
fig.update(frames=frames); 

sliders =  [dict(steps= [dict(method= 'animate',
                                               args= [ [ f'fr{k}'], #this steps refers to the frame of name f'fr{k}
                                               dict(mode= 'immediate',
                                                    frame= dict( duration=100, redraw= True ),
                                                    fromcurrent=True,
                                                    transition=dict( duration= 0))],
                                                label=f"fr{k}") for k, s  in enumerate(t)], #this is the step label on the slider 
                                  minorticklen=0,
                                  x=0,
                                  len=1)]
fig.update_layout(sliders=sliders)

fig.update_layout(updatemenus=[dict(type='buttons', 
                                y=0.2,
                                x=1.05,
                                active=0,
                                buttons=[dict(label='Play',
                                              method='animate',
                                              args=[None, 
                                                    dict(frame=dict(duration=1000, #HERE you can change the frame rate
                                                                    redraw=True),
                                                         transition=dict(duration=0),
                                                         fromcurrent=True,
                                                         mode='afterall')])])]);

#https://plotly.com/python/reference/layout/
#to disable rotation and zooming whilst animation playing,
#also need to hide toolbar, as this overrides False
#show toolbar permanaently when animation paused 
fig.update_layout(dragmode=False)

#Mode mode = 'next' which is the mode of transition between frames. ‘Next’ automatically interpolates the datapoints, meaning we get a gradual effect. Other common options include 'immediate' and 'afterall'
#no difference between 'immediate' and 'afterall' as regards rotating fig with mouse while animating
#can we pause the animation on mouse down and restart it on ouse up?

#Events
#click_data (dict): Click event data from plotly_click event.

#clickannotation_data (dict): Clickannotation event data from plotly_clickannotation event.

#hover_data (dict): Hover event data from plotly_hover and plotly_unhover events.

#relayout_data (dict): Relayout event data from plotly_relayout event

#restyle_data (dict): Restyle event data from plotly_restyle event

#selected_data (dict): Selected event data from plotly_selected and plotly_deselect events.

#viewport (dict): Current viewport state, i.e. the x- and y-axis limits of the displayed plot. Updated on plotly_relayout, plotly_relayouting and plotly_restyle events.

#viewport_update_policy (str, default = ‘mouseup’): Policy by which the viewport parameter is updated during user interactions

#mouseup: updates are synchronized when mouse button is released after panning

#continuous: updates are synchronized continually while panning

#throttle: updates are synchronized while panning, at intervals determined by the viewport_update_throttle parameter

#viewport_update_throttle (int, default = 200, bounds = (0, None)): Time interval in milliseconds at which viewport updates are synchronized when viewport_update_policy is “throttle”.

print("hello")

#fig#.show()
plotly_pane = pn.pane.Plotly(fig,viewport_update_policy='continuous', config={"responsive": True})


@pn.depends(plotly_pane.param.click_data, watch=True)
def _update_after_click_on_1(event):
    # e.g. {‘points’: [{‘curveNumber’: 3, ‘pointNumber’: 5, ‘pointIndex’: 5, ‘x’: 74, ‘y’: 0.6295433958355939}]}
    fig.update_layout(width=900, height=700);
    print("hello click_data")
#    if event.name == 'click_data':
#        if event.obj is not None:
#            print("on_click", event.obj.click_data)
#            print("on_click old", event.old)
#            print("on_click new", event.new)


@pn.depends(plotly_pane.param.relayout_data , watch=True)
def _update_after_relayout_data_1(event):
    # e.g.{"scene.camera": {"up": {"x": 0, "y": 0, "z": 1}, "center": {"x": 0, "y": 0, "z": 0}, "eye": {"x": 0.7520787434923215, "y": 0.7700591365638522, "z": 1.5366803473011377}, "projection": {"type": "perspective"}}}
    fig.update_layout(width=800, height=600);
    print("hello relayout_data ")
#    if event.name == 'click_data':
#        if event.obj is not None:
#            print("on_click", event.obj.click_data)
#            print("on_click old", event.old)
#            print("on_click new", event.new)



@pn.depends(plotly_pane.param.restyle_data , watch=True)
def _update_after_restyle_data_1(event):
    # e.g. {‘points’: [{‘curveNumber’: 3, ‘pointNumber’: 5, ‘pointIndex’: 5, ‘x’: 74, ‘y’: 0.6295433958355939}]}
    fig.update_layout(width=800, height=600);
    print("hello restyle_data ")
#    if event.name == 'click_data':
#        if event.obj is not None:
#            print("on_click", event.obj.click_data)
#            print("on_click old", event.old)
#            print("on_click new", event.new)


@pn.depends(plotly_pane.param.link_figure , watch=True)
def _update_after_link_figure_1(event):
    # e.g. {‘points’: [{‘curveNumber’: 3, ‘pointNumber’: 5, ‘pointIndex’: 5, ‘x’: 74, ‘y’: 0.6295433958355939}]}
    fig.update_layout(width=800, height=600);
    print("hello link_figure ")
#    if event.name == 'click_data':
#        if event.obj is not None:
#            print("on_click", event.obj.click_data)
#            print("on_click old", event.old)
#            print("on_click new", event.new)

@pn.depends(plotly_pane.param.viewport , watch=True)
def _update_after_viewport_1(event):
    # e.g. {‘points’: [{‘curveNumber’: 3, ‘pointNumber’: 5, ‘pointIndex’: 5, ‘x’: 74, ‘y’: 0.6295433958355939}]}
    fig.update_layout(width=800, height=600);
    print("hello viewport")
#    if event.name == 'click_data':
#        if event.obj is not None:
#            print("on_click", event.obj.click_data)
#            print("on_click old", event.old)
#            print("on_click new", event.new)

#plotly_pane.param.watch(on_click, ["click_data"],onlychanged=False)
plotly_pane


hello


hello relayout_data 
hello relayout_data 
hello relayout_data 
hello relayout_data 
hello relayout_data 
hello click_data 
hello click_data 
hello relayout_data 
hello relayout_data 
hello relayout_data 
hello relayout_data 
hello relayout_data 
hello relayout_data 
hello relayout_data 
hello relayout_data 
hello relayout_data

Plotly(Figure, config={'responsive': True}, viewport_update_policy='continuous')

In [40]:
settings_panel = plotly_pane.controls(jslink=True)
settings_panel

Tabs
    [0] WidgetBox(margin=(5, 10), name='Controls')
        [0] StaticText(value='<b>Controls</b>')
        [1] DictInput(description='Plotly configuration o..., name='Config', serializer='json', type=<class 'dict'>, value={'responsive': True})
        [2] DictInput(description='Relayout event d..., name='Relayout data', serializer='json', type=<class 'dict'>, value={'width': 800, ...})
        [3] ListInput(description='Restyle event d..., name='Restyle data', serializer='json', type=<class 'list'>)
        [4] DictInput(description='Current viewport state, ..., name='Viewport', serializer='json', type=<class 'dict'>)
        [5] Select(description='Policy by which the viewp..., name='Viewport update policy', options=OrderedDict({'mouseup': 'm...]), value='continuous')
        [6] IntInput(description='Time interval i..., name='Viewport update t..., start=0, value=200)
    [1] WidgetBox(margin=(5, 10), name='Layout')
        [0] StaticText(value='<b>Layout</b>')
        [1] TextInput(description='String identifier f..., name='Name', value='Plotly00694')
        [2] LiteralInput(description='Whether the object should..., name='Align', serializer='json', value='start')
        [3] LiteralInput(description='Describes the proportiona..., name='Aspect ratio', serializer='json')
        [4] ListInput(description='CSS classes t..., name='Css classes', serializer='json', type=<class 'list'>)
        [5] IntInput(description='The height of the compone..., name='Height', start=0)
        [6] IntInput(description='Minimal width o..., name='Min width', start=0)
        [7] IntInput(description='Minimal height o..., name='Min height', start=0)
        [8] IntInput(description='Minimal width o..., name='Max width', start=0)
        [9] IntInput(description='Minimal height o..., name='Max height', start=0)
        [10] LiteralInput(description='Allows to create addition..., name='Margin', serializer='json', value=(5, 10))
        [11] DictInput(description='Dictionary of C..., name='Styles', serializer='json', type=<class 'dict'>)
        [12] ListInput(description='List of arbitrary t..., name='Tags', serializer='json', type=<class 'list'>)
        [13] IntInput(description='The width of the componen..., name='Width', start=0)
        [14] Select(description='Describes how the compone..., name='Width policy', options=OrderedDict({'auto': 'auto...]), value='auto')
        [15] Select(description='Describes how the compone..., name='Height policy', options=OrderedDict({'auto': 'auto...]), value='auto')
        [16] Select(description='How the component should ..., name='Sizing mode', options=OrderedDict({'fixed': 'fix...]))
        [17] Checkbox(name='Visible', value=True)